In [10]:
import numpy as np
import sqlite3
from scipy import spatial

def save_gbs(dbpath):
    conn = sqlite3.connect(dbpath)
    
    for r in range(6,43,2):
        if r > 6:
            conn.execute('alter table data_%d add column a float'%r)
            conn.execute('alter table data_%d add column b float'%r)
            conn.execute('alter table data_%d add column c float'%r)
            conn.execute('alter table data_%d add column d float'%r)
        for crack_id in range(1,9):
            for arr_index in range(0,234): # This covers all the array indexes
                points = conn.execute('select grain_id,x,y,z,num_xtal from neighbor_%d where crack_id=%d and \
                arr_index=%d order by rank asc'%(r,crack_id,arr_index))
                x_s = []
                y_s = []
                z_s = []
                ids = []
                for point in points:
                    grain_id = int(point[0]) % 10000
                    x = point[1]
                    y = point[2]
                    z = point[3]
                    num_xtal = point[4]
                    x_s.append(x)
                    y_s.append(y)
                    z_s.append(z)
                    ids.append(grain_id)

                if num_xtal > 1 and len(x_s) > 0:
                    tree = spatial.KDTree(zip(x_s,y_s,z_s))
                    cursor = conn.execute('select distinct grain_id%%10000,rank from neighbor_%d where crack_id=%d \
                    and arr_index=%d order by rank limit 2;'%(r,crack_id,arr_index))
                    temp = 1
                    for row in cursor:
                        if temp == 1:
                            grain_id1 = row[0]
                            temp = 2
                        if temp == 2:
                            grain_id2 = row[0]

                    to_fit = []
                    for i in range(len(ids)):
                        if ids[i] == grain_id1 or ids[i] == grain_id2:
                            radius = 7
                            neighbors = tree.query_ball_point((x_s[i],y_s[i],z_s[i]), radius)
                            
                            for j in neighbors:
                                if ids[i]==grain_id1 and ids[j]==grain_id2 or ids[i]==grain_id2 and ids[j]==grain_id1:
                                    if (x_s[i], y_s[i], z_s[i]) not in to_fit:
                                        to_fit.append((x_s[i], y_s[i], z_s[i]))
                                    if (x_s[j], y_s[j], z_s[j]) not in to_fit:
                                        to_fit.append((x_s[j], y_s[j], z_s[j]))
                    if len(to_fit) > 2:
                        pts = np.reshape(to_fit, (len(to_fit),3)).T
                        centroid = np.mean(pts, axis=1) # centroid, plane has to go through here
                        A = np.array(pts)
                        A = A - centroid.reshape((3,1))
                        U, S, V = np.linalg.svd(A)
                        n = U[:,2]
                        a = n[0]
                        b = n[1]
                        c = n[2]
                        d = - centroid.dot(n)

                        # a,b,c,d are set for the gb plane here
                        conn.execute('update data_%d set a=%f, b=%f, c=%f, d=%f where crack_id=%d and arr_index=%d' \
                                    % (r,a,b,c,d,crack_id,arr_index))
        print (r - 6.0) / (42.0 - 6.0)
    conn.commit()
    conn.close()

In [11]:
from math import acos, pi

def save_angle_gb(dbpath):
    conn = sqlite3.connect(dbpath)
    x_center = 0.0
    y_center = 744.0 / 2.0
    z_center = 306.0 # Based on average of Z's
    for r in range(6,43,2):
#         conn.execute('alter table data_%d add column angle_gb float'%r)
        points = conn.execute('select a,b,c,x,y,z,crack_id,arr_index from data_%d'%r)
        for point in points:
            a = point[0]
            b = point[1]
            c = point[2]
            x = point[3]
            y = point[4]
            z = point[5]
            crack_id = point[6]
            arr_index = point[7]
            vec2pt = np.array((x - x_center, y - y_center, z - z_center))
            vec2gb = np.array((a,b,c))
            if a is not None and b is not None and c is not None:
                angle_gb = acos(vec2pt.dot(vec2gb) / ((vec2pt.dot(vec2pt) ** 0.5)*(vec2gb.dot(vec2gb) ** 0.5)))
                if angle_gb > pi / 2.0:
                    angle_gb = abs(angle_gb - pi)
                conn.execute('update data_%d set angle_gb=%f where crack_id=%d and arr_index=%d' \
                             %(r,angle_gb,crack_id,arr_index))
    conn.commit()
    conn.close()

In [12]:
def save_dist_gb(dbpath):
    conn = sqlite3.connect(dbpath)
    for r in range(6,43,2):
        conn.execute('alter table data_%d add column dist_gb float'%r)
        conn.execute('alter table data_%d add column signed_dist_gb float'%r)
        points = conn.execute('select a,b,c,d,x,y,z,crack_id,arr_index from data_%d'%r)
        for point in points:
            a = point[0]
            b = point[1]
            c = point[2]
            d = point[3]
            x = point[4]
            y = point[5]
            z = point[6]
            crack_id = point[7]
            arr_index = point[8]
            if a is not None and b is not None and c is not None:
                signed_dist_gb = (a*x + b*y + c*z + d) / (a**2 + b**2 + c**2)**0.5
                dist_gb = abs(signed_dist_gb)
                conn.execute('update data_%d set dist_gb=%f where crack_id=%d and arr_index=%d' \
                             %(r,dist_gb,crack_id,arr_index))
                conn.execute('update data_%d set signed_dist_gb=%f where crack_id=%d and arr_index=%d' \
                             %(r,signed_dist_gb,crack_id,arr_index))
    conn.commit()
    conn.close()
    print 'done'

In [13]:
# save_gbs('../../big-data-smaller-r/base.db')

In [14]:
# save_angle_gb('../../big-data-smaller-r/base.db')

In [15]:
# save_dist_gb('../../big-data-smaller-r/base.db')